In [42]:
import requests
import pandas as pd
import json
import base64

def obtener_token_vald(client_id, client_secret):
    """Obtiene un token de autenticación de la API de VALD"""
    
    # URL para solicitar el token
    token_url = "https://security.valdperformance.com/connect/token"
    
    # Datos para la solicitud del token
    payload = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }
    
    try:
        # Realizar la solicitud
        response = requests.post(token_url, data=payload)
        
        # Verificar respuesta
        if response.status_code == 200:
            token_data = response.json()
            print("✅ Autenticación exitosa")
            return token_data.get('access_token')
        else:
            print(f"❌ Error en la autenticación: {response.status_code}")
            print(response.text)
            return None
            
    except Exception as e:
        print(f"❌ Error inesperado: {str(e)}")
        return None

def obtener_datos_vald(token, tenant_id, fecha_desde):
    """Obtiene los datos de pruebas desde la API de VALD"""
    
    # URL base según región
    base_url = "https://prd-aue-api-externalnordbord.valdperformance.com"
    
    # Endpoint y parámetros
    endpoint = "/tests/v2"
    params = {
        "tenantId": tenant_id,
        "modifiedFromUtc": fecha_desde
    }
    
    # Headers con token
    headers = {
        "Authorization": f"Bearer {token}"
    }
    
    # Realizar la solicitud
    print("🔄 Solicitando datos a la API...")
    response = requests.get(f"{base_url}{endpoint}", params=params, headers=headers)
    
    # Verificar respuesta
    if response.status_code == 200:
        print(f"✅ Datos obtenidos correctamente")
        return response.json()
    elif response.status_code == 204:
        print("⚠️ No hay más registros para obtener.")
        return []
    else:
        print(f"❌ Error al obtener datos: {response.status_code}")
        print(response.text)
        return None

def procesar_datos(datos):
    """Convierte los datos JSON a un DataFrame de pandas y normaliza columnas anidadas"""
    
    # Verificar tipo de datos y normalizarlos
    if isinstance(datos, dict) and 'items' in datos:
        df = pd.DataFrame(datos['items'])
    elif isinstance(datos, list):
        df = pd.DataFrame(datos)
    else:
        df = pd.DataFrame([datos])
    
    # Si existe la columna 'tests' y contiene datos, normalizarla
    if 'tests' in df.columns and len(df) > 0 and isinstance(df['tests'].iloc[0], (list, dict)):
        try:
            # Normalizar la columna tests
            expanded_df = pd.json_normalize(df['tests'].iloc[0])
            
            # Convertir columnas de fecha a datetime
            date_columns = [col for col in expanded_df.columns 
                            if 'date' in col.lower() or 'time' in col.lower()]
            for col in date_columns:
                expanded_df[col] = pd.to_datetime(expanded_df[col], errors='ignore')
            
            # Reemplazar el DataFrame original con el normalizado
            df = expanded_df
            print("✅ Normalización aplicada a la columna 'tests'")
        except Exception as e:
            print(f"⚠️ No se pudo normalizar la columna 'tests': {e}")
    
    print(f"✅ DataFrame creado con {df.shape[0]} filas y {df.shape[1]} columnas")
    return df

def guardar_csv(df, ruta_archivo):
    """Guardar el DataFrame como CSV"""
    
    df.to_csv(ruta_archivo, index=False, encoding='utf-8-sig')
    print(f"✅ Archivo guardado como: {ruta_archivo}")
    
    # Verificar el archivo guardado
    df_verificacion = pd.read_csv(ruta_archivo)
    print(f"✅ Verificación: {df_verificacion.shape[0]} filas, {df_verificacion.shape[1]} columnas")

# Ejecución principal
if __name__ == "__main__":
    # Configuración
    CLIENT_ID = "Nr37673W7ncT4qBQ=="
    CLIENT_SECRET = "EMiYKddb7wKrwgxxLqopECDpkNLiS1XOEaw="
    TENANT_ID = "f1185650-fb79-44a0-8b4b-b2bf82d28c83"
    FECHA_DESDE = "2023-01-01T00:00:00.000Z"
    
    # Obtener token
    token = obtener_token_vald(CLIENT_ID, CLIENT_SECRET)
    
    if token:
        # Obtener datos
        datos = obtener_datos_vald(token, TENANT_ID, FECHA_DESDE)
        
        if datos:
            # Procesar datos
            df = procesar_datos(datos)
            
            # Guardar como CSV
            guardar_csv(df, "datos_vald_final.csv")
            
            # Mostrar el DataFrame
            print("\n✅ Proceso completado. DataFrame resultante:")
            print(df.head())
    else:
        print("❌ No se pudo continuar sin un token válido.")

✅ Autenticación exitosa
🔄 Solicitando datos a la API...
✅ Datos obtenidos correctamente
✅ Normalización aplicada a la columna 'tests'
✅ DataFrame creado con 50 filas y 20 columnas
✅ Archivo guardado como: datos_vald_final.csv
✅ Verificación: 50 filas, 20 columnas

✅ Proceso completado. DataFrame resultante:
                              profileId                                testId  \
0  9cd86ccc-5359-4737-a6af-5216f9e8059a  710db7b2-a619-4cb9-98e6-998880f6cf83   
1  1b43cb76-9be4-489f-a10c-c0fe58937649  a6f214fb-6a2e-4b8a-8e6d-683512ce8a4b   
2  1b43cb76-9be4-489f-a10c-c0fe58937649  c93cdd8c-0147-4aba-9c27-ae01b06f95b8   
3  e19c69b2-5cc3-477f-9231-4d89e4f214d3  44a2d523-8f30-47cf-8e1f-dc9a47399fbc   
4  ef496913-c955-4e59-827c-81f43fbbae0e  f69ebc7e-487b-4aec-a0ce-c429df22599b   

                   modifiedDateUtc                      testDateUtc  \
0 2023-01-03 21:44:10.564000+00:00 2023-01-03 21:43:43.406000+00:00   
1 2023-01-10 03:48:42.999000+00:00 2023-01-10 03:48:42.416000+

C:\Users\dell\AppData\Local\Temp\ipykernel_11076\705797274.py:92: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  expanded_df[col] = pd.to_datetime(expanded_df[col], errors='ignore')
C:\Users\dell\AppData\Local\Temp\ipykernel_11076\705797274.py:92: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  expanded_df[col] = pd.to_datetime(expanded_df[col], errors='ignore')


In [26]:
def procesar_datos(datos):
    """Convierte los datos JSON a un DataFrame de pandas con estructura normalizada"""
    
    # Verificar tipo de datos y obtener la lista principal
    if isinstance(datos, dict) and 'items' in datos:
        items = datos['items']
    elif isinstance(datos, list):
        items = datos
    else:
        items = [datos]
    
    # Crear DataFrame base
    df = pd.DataFrame(items)
    
    # Expandir columnas que contienen diccionarios o listas
    for columna in df.columns:
        # Si la columna contiene diccionarios, expandirla
        if isinstance(df[columna].iloc[0] if len(df) > 0 and df[columna].iloc[0] is not None else None, dict):
            # Crear un prefijo para evitar conflictos de nombres
            temp_df = pd.json_normalize(df[columna].tolist(), sep='_')
            
            # Renombrar columnas para evitar duplicados
            temp_df = temp_df.add_prefix(f"{columna}_")
            
            # Eliminar columna original y añadir las nuevas
            df = df.drop(columns=[columna])
            df = pd.concat([df, temp_df], axis=1)
    
    # Manejar campos de fecha y convertirlos a datetime
    for col in df.columns:
        if isinstance(col, str) and ('date' in col.lower() or 'time' in col.lower()):
            try:
                df[col] = pd.to_datetime(df[col], errors='ignore')
            except:
                pass  # Si no se puede convertir, dejarlo como está
    
    print(f"✅ DataFrame creado con {df.shape[0]} filas y {df.shape[1]} columnas")
    return df



In [34]:
df

,tests
0,[{'profileId': '9cd86ccc-5359-4737-a6af-5216f9...


In [39]:
# Normalize the DataFrame by expanding the nested dictionary in 'tests' column
def normalize_df():
    # Expand the tests column which contains the list of dictionaries
    expanded_df = pd.json_normalize(df['tests'].iloc[0])
    
    # Convert date columns to datetime
    date_columns = [col for col in expanded_df.columns if 'date' in col.lower() or 'time' in col.lower()]
    for col in date_columns:
        expanded_df[col] = pd.to_datetime(expanded_df[col])
    
    # Display info about the new DataFrame
    print("\n✅ DataFrame normalizado:")
    print(f"Filas: {expanded_df.shape[0]}, Columnas: {expanded_df.shape[1]}")
    
    return expanded_df

# Use the function
normalized_df = normalize_df()


✅ DataFrame normalizado:
Filas: 50, Columnas: 20


In [40]:
normalized_df

,profileId,testId,modifiedDateUtc,testDateUtc,testTypeId,testTypeName,notes,device,leftAvgForce,leftImpulse,leftMaxForce,leftTorque,leftCalibration,leftRepetitions,rightAvgForce,rightImpulse,rightMaxForce,rightTorque,rightCalibration,rightRepetitions
0,9cd86ccc-5359-4737-a6af-5216f9e8059a,710db7b2-a619-4cb9-98e6-998880f6cf83,2023-01-03 21:44:10.564000+00:00,2023-01-03 21:43:43.406000+00:00,596a6004-553f-4afa-aef0-1474903b462e,Nordic,,NordBord-1756,248.333333,2043.020,305.00,119.407500,0.0,3,256.666667,1972.205,312.50,122.343750,0.0,3
1,1b43cb76-9be4-489f-a10c-c0fe58937649,a6f214fb-6a2e-4b8a-8e6d-683512ce8a4b,2023-01-10 03:48:42.999000+00:00,2023-01-10 03:48:42.416000+00:00,f9755ba4-17cf-4c47-9c3d-a8995905073d,ISO Prone,,NordBord-1613,197.666667,1345.830,207.50,71.898750,0.0,3,191.416667,1286.900,197.50,68.433750,0.0,3
2,1b43cb76-9be4-489f-a10c-c0fe58937649,c93cdd8c-0147-4aba-9c27-ae01b06f95b8,2023-01-10 03:50:12.651000+00:00,2023-01-10 03:50:12.526000+00:00,1ccf7d5f-307d-4d1f-9fb3-1d8a9878d94a,ISO 30,,NordBord-1613,158.916667,720.375,168.50,58.385250,0.0,3,190.833333,922.815,203.50,70.512750,0.0,3
3,e19c69b2-5cc3-477f-9231-4d89e4f214d3,44a2d523-8f30-47cf-8e1f-dc9a47399fbc,2023-01-10 03:56:18.330000+00:00,2023-01-10 03:56:18.009000+00:00,f9755ba4-17cf-4c47-9c3d-a8995905073d,ISO Prone,,NordBord-1613,310.833333,2088.030,319.75,133.815375,0.0,3,346.000000,2669.200,348.75,145.951875,0.0,3
4,ef496913-c955-4e59-827c-81f43fbbae0e,f69ebc7e-487b-4aec-a0ce-c429df22599b,2023-01-30 22:49:42.086000+00:00,2023-01-30 22:49:02.345000+00:00,596a6004-553f-4afa-aef0-1474903b462e,Nordic,,NordBord-1756,267.416667,3067.100,283.50,121.196250,0.0,3,299.416667,3058.610,315.25,134.769375,0.0,3
5,bb261a4b-b30f-4dc9-ad2b-e94d04d6ae30,4da5233b-3e47-469a-8e9a-5d1fa79684f9,2023-01-30 22:50:49.045000+00:00,2023-01-30 22:50:47.377000+00:00,596a6004-553f-4afa-aef0-1474903b462e,Nordic,,NordBord-1756,433.833333,5972.360,472.25,180.635625,0.0,3,450.750000,6136.685,476.75,182.356875,0.0,3
6,95bca20b-7ec7-4d4c-861c-3c5ec38803e0,b62232b7-bd7b-4ebd-9618-9cc896462745,2023-01-30 22:52:56.858000+00:00,2023-01-30 22:52:53.674000+00:00,596a6004-553f-4afa-aef0-1474903b462e,Nordic,,NordBord-1756,321.437500,6771.640,450.25,200.586375,0.0,4,271.000000,7193.225,329.50,146.792250,0.0,3
7,94e4c6d0-cabc-4113-8f6f-d7ed78c00344,0afb2ec8-4cf7-40b6-9f3d-de71dacd9723,2023-01-30 22:55:57.826000+00:00,2023-01-30 22:55:04.162000+00:00,596a6004-553f-4afa-aef0-1474903b462e,Nordic,,NordBord-1756,286.375000,4428.205,291.50,111.498750,0.0,2,301.750000,4375.305,312.25,119.435625,0.0,2
8,d7bfa483-dda7-404a-98c0-8316934e9b17,69287d0a-16a3-4839-ab06-12bfefe7dcd2,2023-01-30 22:56:58.187000+00:00,2023-01-30 22:56:44.378000+00:00,596a6004-553f-4afa-aef0-1474903b462e,Nordic,,NordBord-1756,382.000000,4147.845,395.00,158.197500,0.0,3,358.500000,4393.790,371.00,148.585500,0.0,3
9,2929d7f7-8683-4bfd-97db-cd33ae64f6d9,ff8321df-59d7-46e2-b65e-7e00717847fb,2023-01-30 22:58:58.811000+00:00,2023-01-30 22:58:54.558000+00:00,596a6004-553f-4afa-aef0-1474903b462e,Nordic,,NordBord-1756,461.250000,5630.920,481.00,214.285500,0.0,3,402.583333,5621.600,411.00,183.100500,0.0,3
